## Student Performance Indicator

### Life cycle of a Machine Learning Project:

- Understading the problem statement
- Data Collection
- Data description and sanity checks
- Exploratory Data Analysis
- Pre-Processing
- Model Training
- Best model selection

### 1) Problem statement

- This project understands how the student's performance is affected by other variables such as Gender, Ethnicity, Parental level of education, Lunch and Test preparation course.

### 2) Data Collection

- Dataset source: https://www.kaggle.com/datasets/spscientist/students-performance-in-exams?datasetId=74977
- The data consists of 8 columns and 1000 rows.

#### 2.1) Importing required libraries

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

**Import the CSV Data as a Pandas Dataframe**

In [6]:
df_students = pd.read_csv('../data/raw/stud.csv')

In [7]:
# Show top 5 records
df_students.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [8]:
# Dataset shape
df_students.shape

(1000, 8)

#### 2.2) Dataset Info

- gender: sex of the students (male/female)
- race ethnicity: ethnicity of the students (determined by groups)
- parental level of education: parent's final level of education (bachelor's degree, associate's degree, hight school, etc)
- lunch: having lunch before test (standard or free/reduced)
- test preparation course: completed or not completed the course before test
- actual math score
- actual reading score
- actual writing score

### 3) Data description and sanity checks

**What to perform here?**

- Check missing values
- Check duplicated values
- Check data types
- Check descriptive statistics for the numerical and categorical columns
- Check some unique values of each column

#### 3.1) Check missing values

In [9]:
df_students.isna().sum()

gender                         0
race_ethnicity                 0
parental_level_of_education    0
lunch                          0
test_preparation_course        0
math_score                     0
reading_score                  0
writing_score                  0
dtype: int64

**There are no missing values on the dataset**

#### 3.2) Check duplicated values

In [10]:
df_students.duplicated().sum()

0

**There are no duplicated value in the dataset**

#### 3.3) Check data types

In [12]:
df_students.dtypes # or df_students.info()

gender                         object
race_ethnicity                 object
parental_level_of_education    object
lunch                          object
test_preparation_course        object
math_score                      int64
reading_score                   int64
writing_score                   int64
dtype: object

#### 3.4) Descriptive statistics

In [13]:
df_numerical = df_students.select_dtypes(exclude='object')
df_categorical = df_students.select_dtypes(include='object')

##### 3.4.1) Numerical

##### 3.4.2) Categorial